In [1]:
import numpy as np
import tensorflow as tf #OLD VERSION ##########UPDATE THIS##########
from othello import Board
from copy import deepcopy

def reset_graph():
    if("sess" in globals() and sess):
        sess.close()
    tf.reset_default_graph()

def buildGraph(inputDim, dataType, hLayersDim, name="player1"):
    with tf.variable_scope(name) as scope:
        x = tf.placeholder(dataType, [None, inputDim])

        layers = []
        biases = []

        # for i, dim in enumerate(hLayersDim):
        #     if i == 0:
        #         layers.append(tf.Variable("W"+str(i), dataType, [inputDim, dim]))
        #     elif i == len(hLayersDim) - 1:
        #         layers.append(tf.Variable("W"+str(i), dataType, [dim, inputDim]))
        #     else:
        #         layers.append(tf.Variable("W"+str(i), dataType, [hLayersDim[i-1], dim]))
        #     biases.append(tf.Variable("b"+str(i), dataType, [dim]))

        # for i, dim in enumerate(hLayersDim):
        #     activations.append(tf.nn.relu())

        layers.append(tf.get_variable("W0", [inputDim, hLayersDim[0]], dataType, initializer=tf.contrib.layers.xavier_initializer()))
        layers.append(tf.get_variable("W1", [hLayersDim[0], hLayersDim[1]], dataType, initializer=tf.contrib.layers.xavier_initializer()))
        layers.append(tf.get_variable("W2", [hLayersDim[1], hLayersDim[2]], dataType, initializer=tf.contrib.layers.xavier_initializer()))
        layers.append(tf.get_variable("W3", [hLayersDim[2], inputDim], dataType, initializer=tf.contrib.layers.xavier_initializer()))

        biases.append(tf.get_variable("b0", [hLayersDim[0]], dataType, initializer=tf.contrib.layers.xavier_initializer()))
        biases.append(tf.get_variable("b1", [hLayersDim[1]], dataType, initializer=tf.contrib.layers.xavier_initializer()))
        biases.append(tf.get_variable("b2", [hLayersDim[2]], dataType, initializer=tf.contrib.layers.xavier_initializer()))
        biases.append(tf.get_variable("b3", [inputDim], dataType, initializer=tf.contrib.layers.xavier_initializer()))

        hiddenStates = []

        hiddenStates.append(tf.nn.relu(tf.matmul(x, layers[0]) + biases[0])) #Olhar o relu6, pode ser melhor
        hiddenStates.append(tf.nn.relu(tf.matmul(hiddenStates[-1], layers[1]) + biases[1])) #Olhar o relu6, pode ser melhor
        hiddenStates.append(tf.nn.relu(tf.matmul(hiddenStates[-1], layers[2]) + biases[2])) #Olhar o relu6, pode ser melhor
        hiddenStates.append(tf.nn.softmax(tf.matmul(hiddenStates[-1], layers[3]) + biases[3])) #Olhar o relu6, pode ser melhor

        output = hiddenStates[-1] #sparse_softmax_cross_entropy_with_logits

        return {
            "name": name,
            "x": x,
            "layers": layers,
            "biases": biases,
            "hiddenStates": hiddenStates,
            "output": output
        }
    
def predict(agent, state):
    return sess.run(agent["output"], feed_dict={agent["x"]: state})

def makeMove(agent, board):
    if(agent["name"] == "BLACK"):
        player = 1
        tempBoard = board.board
    else:
        player = -1
        tempBoard = board.inverted_board()
       
    moves = board.possible_moves(player) 
    if(len(moves) == 0):
        return (-1, player)
    probs = predict(agent, tempBoard.reshape([1,-1])).squeeze()
    
    new_probs = np.zeros(boardDim)
    for x, y, _ in moves:
        new_probs[x*8 + y] = probs[x*8 + y]
        
    new_probs = new_probs/np.sum(new_probs)
    
    selected_move = np.random.choice(boardDim, 1, p=new_probs)[0]
    
    board.move(selected_move//8, selected_move%8, player)
    
    return (selected_move, player)
        
def maybePrint(shouldI, s):
    if(shouldI):
        print(s)
    
def playGame(board, gamma, v = False):
    boardHistoryP1 = []
    boardHistoryP2 = []
    moveSequenceP1 = []
    moveSequenceP2 = []
    
    maybePrint(v, b)
    while(not b.finished()):
        boardBeforeMoveP1 = deepcopy(board.board)
        P1Move = makeMove(p1, b)[0]
        if(P1Move != -1):
            boardHistoryP1.append(boardBeforeMoveP1)
            moveSequenceP1.append(P1Move)
        
        boardBeforeMoveP2 = deepcopy(board.inverted_board())
        P2Move = makeMove(p2, b)[0]
        if(P2Move != -1):
            boardHistoryP2.append(boardBeforeMoveP2)
            moveSequenceP2.append(P1Move)
        maybePrint(v, "====================")
        maybePrint(v, (moveSequenceP1[-1]//8, moveSequenceP1[-1]%8))
        maybePrint(v, b)
        maybePrint(v, "====================")
        maybePrint(v, (moveSequenceP2[-1]//8, moveSequenceP2[-1]%8))
        maybePrint(v, b)
    maybePrint(v, b.score())
    
    rewardSequenceP1 = []
    rewardSequenceP2 = []
    r = reward(board)
    
    for i in range(len(moveSequenceP1)):
        rewardSequenceP1.append(r* (gamma**i))
    
    for i in range(len(moveSequenceP2)):
        rewardSequenceP2.append(-r* (gamma**i))
        
    rewardSequenceP1 = np.flip((np.array(rewardSequenceP1)), 0)
    rewardSequenceP2 = np.flip((np.array(rewardSequenceP2)), 0)
    return ((boardHistoryP1, moveSequenceP1, rewardSequenceP1), (boardHistoryP2, moveSequenceP2, rewardSequenceP2))
    
def reward(board):
    if(np.sum(board.board) > 0):
        return 1
    elif(np.sum(board.board) < 0):
        return -1
    else:
        return 0

reset_graph()

gamma = 0.99
boardDim = 8*8
hLayersDim = [128, 256, 128]
b = Board()
p1 = buildGraph(boardDim, tf.float32, hLayersDim, "BLACK")
p2 = buildGraph(boardDim, tf.float32, hLayersDim, "WHITE")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    print(playGame(b, gamma, True))


. . . . . . . . 
. . . . . . . . 
. . . . . . . . 
. . . □ ■ . . . 
. . . ■ □ . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 

(3, 2)
. . . . . . . . 
. . . . . . . . 
. . . . □ . . . 
. . ■ ■ □ . . . 
. . . ■ □ . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 

(3, 2)
. . . . . . . . 
. . . . . . . . 
. . . . □ . . . 
. . ■ ■ □ . . . 
. . . ■ □ . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 

(3, 5)
. . . . . . . . 
. . . . . . . . 
. . . . □ . . . 
. . ■ □ ■ ■ . . 
. . □ □ □ . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 

(3, 5)
. . . . . . . . 
. . . . . . . . 
. . . . □ . . . 
. . ■ □ ■ ■ . . 
. . □ □ □ . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 

(1, 4)
. . . . □ . . . 
. . . . □ . . . 
. . . . □ . . . 
. . ■ □ □ ■ . . 
. . □ □ □ . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 

(1, 4)
. . . . □ . . . 
. . . . □ . . . 
. . . . □ . . . 
. . ■ □ □ ■ . . 
. . □ □ □ . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 
